In [1]:
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.visualization.petri_net import visualizer as pn_visualizer
import pm4py
import sys
sys.path.append('../') 
from utils.utilities import Utilities
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import time
from IPython.display import clear_output

wilcoxon_signed_rank_test = Utilities.wilcoxon_signed_rank_test


Import data

In [2]:
file_path = '../data/PermitLog.xes'
log = pm4py.read_xes(file_path, progress=False)

parsing log, completed traces ::   0%|          | 0/7065 [00:00<?, ?it/s]

# Baseline

### Alpha miner

In [3]:
net, initial_marking, final_marking = alpha_miner.apply(log)
alpha_model_baseline = (net, initial_marking, final_marking)
fitness_score = Utilities.fitness_score(log, alpha_model_baseline)
mixed_score = Utilities.mixed_score(log, alpha_model_baseline, verbose=True)

print(f'Fitness score: {fitness_score}')
print(f'Mixed score: {mixed_score}')

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

Fitness:  {'perc_fit_traces': 0.0, 'average_trace_fitness': 0.4175010080427763, 'log_fitness': 0.41989689325938545, 'percentage_of_fitting_traces': 0.0}
Simplicity:  0.41904761904761906
Generalization:  0.8552671808747483
Fitness score: 0.41989689325938545
Mixed score: 0.5647372310605843


### Heuristic miner 

In [4]:
net, initial_marking, final_marking = pm4py.discover_petri_net_heuristics(log)
heuristic_model_baseline = (net, initial_marking, final_marking)
fitness_score = Utilities.fitness_score(log, heuristic_model_baseline)
mixed_score = Utilities.mixed_score(log, heuristic_model_baseline, verbose=True)

print(f'Fitness score: {fitness_score}')
print(f'Mixed score: {mixed_score}')

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

Fitness:  {'perc_fit_traces': 0.6510969568294409, 'average_trace_fitness': 0.8911335458231742, 'log_fitness': 0.8910248238210718, 'percentage_of_fitting_traces': 0.6510969568294409}
Simplicity:  0.4786450662739322
Generalization:  0.5947986582652351
Fitness score: 0.8910248238210718
Mixed score: 0.6548228494534131


### Inductive miner

In [5]:
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log)
inductive_model_baseline = (net, initial_marking, final_marking)
fitness_score = Utilities.fitness_score(log, inductive_model_baseline)
mixed_score = Utilities.mixed_score(log, inductive_model_baseline, verbose=True)

print(f'Fitness score: {fitness_score}')
print(f'Mixed score: {mixed_score}')

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

Fitness:  {'perc_fit_traces': 100.0, 'average_trace_fitness': 1.0, 'log_fitness': 1.0, 'percentage_of_fitting_traces': 100.0}
Simplicity:  0.5946547884187082
Generalization:  0.8616797985629043
Fitness score: 1.0
Mixed score: 0.8187781956605376


# Hypeopt

In [6]:
budget_time, start_time, best_model, best_score = None, None, None, None
def check_is_time_limit_reached():
    
    if budget_time == None:
        return
    
    if time.time() - start_time > budget_time:
        print("Time limit reached")
        raise SystemExit


def get_net(params):    
    model_name = params['model_name']
    del params["model_name"]

    if model_name == 'alpha':
        net, initial_marking, final_marking = alpha_miner.apply(log)
    elif model_name == 'alpha_plus':
        net, initial_marking, final_marking =  alpha_miner.apply(log, variant=alpha_miner.Variants.ALPHA_VERSION_PLUS)
    elif model_name == 'inductive':
        net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(log, **params)
    elif model_name == 'heuristic':
        net, initial_marking, final_marking = pm4py.discover_petri_net_heuristics(log, **params)
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    return (net, initial_marking, final_marking)

def objective(params):
    global best_model, best_score
    check_is_time_limit_reached()
    clear_output()
    score_type = params['score_type']
    del params["score_type"]

    model = get_net(params)
    
    if score_type == 'fitness':
        score = Utilities.fitness_score(log, model)
    elif score_type == 'mixed':
        score = Utilities.mixed_score(log, model)
    else:
        raise ValueError(f"Unknown score type: {score_type}")

    if not best_score or score > best_score:
        best_score = score
        best_model = model
    return -score

#### Search space

In [7]:
def create_search_space(eval_func_name):
    search = [
        {
            "model_name": "alpha",
            "score_type": eval_func_name
        },
        {
            "model_name": "inductive",
            "score_type": eval_func_name,
            "noise_threshold": hp.uniform("noise_threshold", 0.0, 0.5),
        },
        {
            "model_name": "heuristic",
            "score_type": eval_func_name,
            "dependency_threshold": hp.uniform("dependency_threshold", 0.0, 0.9),
            "and_threshold": hp.uniform("and_threshold", 0.0, 0.9),
            "loop_two_threshold": hp.uniform("loop_two_threshold", 0.0, 0.9),
        }
    ]
    return hp.choice("classifier_type", search)


## Fitness

In [8]:
budget_time = 1 * 60 # 15 minutes
trials = Trials()
search_space = create_search_space('fitness')
start_time = time.time()
best_hyperparams_fitness = fmin(fn=objective,
                        space=search_space,
                        algo=tpe.suggest,
                        max_evals=1000,
                        trials=trials)

print("Best Hyperparameters:", best_hyperparams_fitness)



replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

Time limit reached                                                                    
  1%|          | 11/1000 [01:03<1:35:10,  5.77s/trial, best loss: -0.9183239520668016]


SystemExit: 

c:\Users\Noel Nathan\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
best_model_fitness = best_model

### Wilcoxon Test

In [11]:

statistic, p_value = wilcoxon_signed_rank_test(log, inductive_model_baseline, best_model_fitness, "fitness")


replaying log with TBR, completed traces ::   0%|          | 0/12 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/12 [00:00<?, ?it/s]


Wilcoxon Signed-Rank Test (Log Loss):
Statistic: 0.0000, P-value: 0.0000
The optimized pipeline significantly beats the baseline.


## Mixed Score

In [12]:
best_model, best_score = None, None

budget_time = 1 * 60 # 15 minutes
trials = Trials()
search_space = create_search_space('mixed')
start_time = time.time()
best_hyperparams_mixed = fmin(fn=objective,
                        space=search_space,
                        algo=tpe.suggest,
                        max_evals=1000,
                        trials=trials)

print("Best Hyperparameters:", best_hyperparams_mixed)

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/1478 [00:00<?, ?it/s]

Time limit reached                                                                   
  0%|          | 4/1000 [01:13<5:04:13, 18.33s/trial, best loss: -0.6499933705388616]


SystemExit: 

c:\Users\Noel Nathan\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
best_model_mixed = best_model

### Wilcoxon test

In [14]:
statistic, p_value = wilcoxon_signed_rank_test(log, inductive_model_baseline, best_model_mixed, "fitness")

replaying log with TBR, completed traces ::   0%|          | 0/12 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/12 [00:00<?, ?it/s]


Wilcoxon Signed-Rank Test (Log Loss):
Statistic: 0.0000, P-value: 0.0000
The optimized pipeline significantly beats the baseline.
